In [2]:
import json
import os

import numpy as np
import torch
import pandas as pd
import trainUtils
from tqdm import tqdm

In [6]:
from esm.models.esmc import ESMC
import numpy as np
import ioutils
from esm.sdk.api import ESMProtein, LogitsConfig
import warnings
warnings.filterwarnings("ignore")
model = ESMC.from_pretrained("esmc_600m", torch.device("cpu"))

In [ ]:
allres = []
bar = tqdm(os.listdir("/data/tyfei/datasets/NCBI_virus/genbank_csv/"))
for file in bar:
    if not file.endswith(".csv"):
        continue
    
    # print(file)
    df = pd.read_csv(f"/data/tyfei/datasets/NCBI_virus/genbank_csv/{file}")
    df = df[df["translation"].notnull()]
    if len(df) == 0:
        continue
    df = df[df["translation"].str.len() > 15]
    df = df[df["translation"].str.len() < 2000]
    bar.set_description(f"Processing item {file} with {len(df)} sequences")
    
    # print(file, len(df))
    
    for i, r in df.iterrows():
        # if pd.isnull(r["translation"]):
        #     # print(i)
        #     continue
        # if len(r["translation"]) < 16:
        #     # print(i)
        #     continue
        res = {}
        protein = ESMProtein(sequence=r["translation"])
        res_s = model.encode(protein)
        res["prot"] = res_s.sequence.cpu().numpy() 
        res["accession"] = r["accession"]
        res["id"] = r["protein_id"]
        res["bin"] = 0 
        res["days"] = 0
        
        allres.append(res)
# for i, r in df.iterrows():
#     if pd.isnull(r["translation"]):
#         # print(i)
#         continue
#     if len(r["translation"]) < 16:
#         # print(i)
#         continue
#     res = {}
#     protein = ESMProtein(sequence=r["translation"])
#     res_s = model.encode(protein)
#     res["prot"] = res_s.sequence.cpu().numpy() 
#     res["accession"] = r["accession"]
#     res["protein_id"] = r["protein_id"]
#     allres.append(res)
    


Processing item NC_055438.1.csv with 1 sequences:   0%|          | 0/18756 [00:00<?, ?it/s]

Processing item NC_015655.1.csv with 5 sequences: 100%|██████████| 18756/18756 [04:10<00:00, 74.84it/s]    


In [8]:
import pickle
with open("/data2/tyfei/virusDataset/proteins.pkl", "wb") as f:
    pickle.dump(allres, f)

In [9]:
len(allres)

678146

In [3]:
df = pd.read_csv("/data2/tyfei/virusDataset/20250318_all_virusDataset.csv", index_col=0)
df.head()

,Unnamed: 0,sseqid,NR_sseqid,taxid,gi,Domain,Realm,Kingdom,Phylum,Subphylum,Class,Order,Family,Genus,Species
0,127890,NP_040995,NP_040995.1,10406,9626717,Viruses,Riboviria,Pararnavirae,Artverviricota,Revtraviricetes,Blubervirales,Hepadnaviridae,Orthohepadnavirus,Ground squirrel hepatitis virus,NaN
15,127905,AAA36008,AAA36008.1,10407,386784,Viruses,Riboviria,Pararnavirae,Artverviricota,Revtraviricetes,Blubervirales,Hepadnaviridae,Orthohepadnavirus,Hepatitis B virus,NaN
77,127967,AAB20084,AAB20084.1,10407,237356,Viruses,Riboviria,Pararnavirae,Artverviricota,Revtraviricetes,Blubervirales,Hepadnaviridae,Orthohepadnavirus,Hepatitis B virus,NaN
78,127968,AAB20085,AAB20085.1,10407,237357,Viruses,Riboviria,Pararnavirae,Artverviricota,Revtraviricetes,Blubervirales,Hepadnaviridae,Orthohepadnavirus,Hepatitis B virus,NaN
81,127971,AAB27982,AAB27982.1,10407,409992,Viruses,Riboviria,Pararnavirae,Artverviricota,Revtraviricetes,Blubervirales,Hepadnaviridae,Orthohepadnavirus,Hepatitis B virus,NaN


In [1]:
import pickle 
with open("/data2/tyfei/COVID/20241209/allprot1204/Germany/data2.pkl", 'rb') as f:
    d = pickle.load(f)

In [2]:
d[0]

{'E': array([ 0, 20, 19,  8, 18,  7,  8,  9,  9, 11,  6, 11,  4, 12,  7, 17,  8,
         7,  4,  4, 18,  4,  5, 18,  7,  7, 18,  4,  4,  7, 11,  4,  5, 12,
         4, 11,  5,  4, 10,  4, 23,  5, 19, 23, 23, 17, 12,  7, 17,  7,  8,
         4,  7, 15, 14,  8, 18, 19,  7, 19,  8, 10,  7, 15, 17,  4, 17,  8,
         8, 10,  7, 14, 13,  4,  4,  7,  2]),
 'M': array([ 0, 20,  5, 13,  8, 17,  6, 11, 12, 11,  7,  9,  9,  4, 15, 15,  4,
         4,  9, 16, 22, 17,  4,  7, 12,  6, 18,  4, 18,  4, 11, 22, 12, 23,
         4,  4, 16, 18,  5, 19,  5, 17, 10, 17, 10, 18,  4, 19, 12, 12, 15,
         4, 12, 18,  4, 22,  4,  4, 22, 14,  7, 11,  4,  5, 23, 18,  7,  4,
         5,  5,  7, 19, 10, 12, 17, 22, 12, 11,  6,  6, 12,  5, 12,  5, 20,
         5, 23,  4,  7,  6,  4, 20, 22,  4,  8, 19, 18, 12,  5,  8, 18, 10,
         4, 18,  5, 10, 11, 10,  8, 20, 22,  8, 18, 17, 14,  9, 11, 17, 12,
         4,  4, 17,  7, 14,  4, 21,  6, 11, 12,  4, 11, 10, 14,  4,  4,  9,
         8,  9,  4,  7, 12,  6, 

In [2]:
with open(os.path.join("/home/tyfei/evoModel/checkpoints/backbone_lr/", "config.json"), "r") as f:
    configs = json.load(f)

In [3]:
pretrain_model = trainUtils.loadPretrainModel(configs)
model = trainUtils.buildModel(configs, pretrain_model, None) 

In [4]:
a = torch.load("/home/tyfei/evoModel/checkpoints/backbone_lr/last.ckpt")
model.load_state_dict(a["state_dict"], strict=False)

_IncompatibleKeys(missing_keys=['esm_model.encoder.sequence_embed.weight', 'esm_model.encoder.plddt_projection.weight', 'esm_model.encoder.plddt_projection.bias', 'esm_model.encoder.structure_per_res_plddt_projection.weight', 'esm_model.encoder.structure_per_res_plddt_projection.bias', 'esm_model.encoder.structure_tokens_embed.weight', 'esm_model.encoder.ss8_embed.weight', 'esm_model.encoder.sasa_embed.weight', 'esm_model.encoder.function_embed.0.weight', 'esm_model.encoder.function_embed.1.weight', 'esm_model.encoder.function_embed.2.weight', 'esm_model.encoder.function_embed.3.weight', 'esm_model.encoder.function_embed.4.weight', 'esm_model.encoder.function_embed.5.weight', 'esm_model.encoder.function_embed.6.weight', 'esm_model.encoder.function_embed.7.weight', 'esm_model.encoder.residue_embed.weight', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.0.weight', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.0.bias', 'esm_model.transformer.blocks.0.attn.layernorm_qkv.1.linear.w

In [5]:
model.only_embed = True
model.tf = -1.0

In [6]:
import pickle 
with open("/data/tyfei/datasets/covid/S_aln.pkl", "rb") as f:
    S = pickle.load(f)
with open("/data/tyfei/datasets/covid/NSP5_aln.pkl", "rb") as f:
    NSP5 = pickle.load(f)
with open("/data/tyfei/datasets/covid/E_aln.pkl", "rb") as f:
    E = pickle.load(f)
    

In [22]:
S[0]["seq_t"] = torch.tensor(S[0]["seq_t"])

In [7]:
from tqdm import tqdm
cnt = 0
res = []
with torch.no_grad():
    model = model.cuda(7)
    for s, e, nsp5 in tqdm(zip(S, E, NSP5)):
        ret = {}
        s["seq_t"] = torch.tensor(s["seq_t"]).cuda(7)
        ret["S"] = model(s).squeeze().cpu().numpy() 
        e["seq_t"] = torch.tensor(e["seq_t"]).cuda(7)
        ret["E"] = model(e).squeeze().cpu().numpy() 
        nsp5["seq_t"] = torch.tensor(nsp5["seq_t"]).cuda(7)
        ret["NSP5"] = model(nsp5).squeeze().cpu().numpy() 
        res.append(ret) 
        cnt += 1 
        if (cnt % 1000) == 0:
            break
        

999it [06:11,  2.69it/s]


In [8]:
with open("/data/tyfei/datasets/covid/teststage2_1000.pkl", "wb") as f:
    pickle.dump(res, f)

In [23]:
model(S[0]).shape

torch.Size([1, 256])